# Data Engineering - Uploading Local File to GBQ

In [26]:
import os
import re
import datetime 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_gbq
import janitor

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

### GBQ Set Up

In this next section I'll connect to my GBQ project and list the data sets inside to test the connection.

In [27]:
# These first two values will be different on your machine. 
service_path = ""  # change this to your file path
service_file = '' # change this to your authentication information  
gbq_proj_id = '' # change this to your project. 

# And this should stay the same. 
private_key =service_path + service_file

In [28]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [29]:
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [30]:
for item in client.list_datasets() : 
    print(item.full_dataset_id)

msba-project-2022:dram_shop
msba-project-2022:wedge_dataset


# Checking for Existing Table

To make sure the data funnel is idempotent, I'll check my project in GBQ for tables that match the required table id, and delete if necessary to avoid duplicates.

In [31]:
# Your code here

table_name = 'dram_items_2022'
#first let's check if the table exists

dataset_name = 'dram_shop'

table_list = client.list_tables(dataset_name)

#now write a loop to iterate over all the values in table_list
#print out the results (tables), if dram_items_2022 is in the result we know it exists.

for table in table_list:
    print(table_id)




msba-project-2022.dram_shop.dram_items_2022
msba-project-2022.dram_shop.dram_items_2022
msba-project-2022.dram_shop.dram_items_2022
msba-project-2022.dram_shop.dram_items_2022


In [32]:
## delete table to keep this function idempotent here (still need to assign table name)

#the next two lines create my table id in gbq format
table_id = ".".join([gbq_proj_id,dataset_name,table_name])

print(table_id)

#this line deletes my table out of gbq if it exists

client.delete_table(table_id, not_found_ok=True)

print(f"Deleted {table_id}.")

msba-project-2022.dram_shop.dram_items_2022
Deleted msba-project-2022.dram_shop.dram_items_2022.


# Reading in Local Data


In [33]:
#First I need to read in the file, using the pandas function creates a dataframe for me.

dramitems_df = pd.read_csv('items-2022-01-01-2022-10-01.csv')
#print(dramitems_df)




/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_2267/3835684965.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dramitems_df = pd.read_csv('items-2022-01-01-2022-10-01.csv')


In [34]:
#show us the data types of columns above (objects)
dramitems_df.dtypes

Date                      object
Time                      object
Time Zone                 object
Category                  object
Item                      object
Qty                        int64
Price Point Name          object
SKU                       object
Modifiers Applied        float64
Gross Sales               object
Discounts                 object
Net Sales                 object
Tax                       object
Transaction ID            object
Payment ID                object
Device Name               object
Notes                     object
Details                   object
Event Type                object
Location                  object
Dining Option            float64
Customer ID               object
Customer Name             object
Customer Reference ID     object
Unit                      object
Count                      int64
Itemization Type          object
dtype: object

# Formatting the Data

In [35]:
#first change dollar sign fields to numeric (gross sales, discount, net sales, tax)
dramitems_df['Gross Sales'] = dramitems_df["Gross Sales"].str.replace('$', '').str.replace(',','')
dramitems_df['Gross Sales'] = pd.to_numeric(dramitems_df['Gross Sales'])

dramitems_df['Discounts'] = dramitems_df["Discounts"].str.replace('$', '').str.replace(',','')
dramitems_df['Discounts'] = pd.to_numeric(dramitems_df['Discounts'])

dramitems_df['Net Sales'] = dramitems_df["Net Sales"].str.replace('$', '').str.replace(',','')
dramitems_df['Net Sales'] = pd.to_numeric(dramitems_df['Net Sales'])

dramitems_df['Tax'] = dramitems_df["Tax"].str.replace('$', '').str.replace(',','')
dramitems_df['Tax'] = pd.to_numeric(dramitems_df['Tax'])

/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_2267/2369041351.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dramitems_df['Gross Sales'] = dramitems_df["Gross Sales"].str.replace('$', '').str.replace(',','')
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_2267/2369041351.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dramitems_df['Discounts'] = dramitems_df["Discounts"].str.replace('$', '').str.replace(',','')
/var/folders/2x/4hk6jhf96_q696_2h9d_fstw0000gp/T/ipykernel_2267/2369041351.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* 

In [36]:
# Now change modifiiers applies float 64 to string

dramitems_df['Modifiers Applied'] = dramitems_df['Modifiers Applied'].astype('str')

In [37]:
## now fill sku with empty strings
dramitems_df['SKU'] = dramitems_df['SKU'].replace(np.nan, '', regex=True)


In [38]:
#Clean up the names with py janitor final step

dramitems_clean = janitor.clean_names(dramitems_df)

# Upoading to GBQ

In [40]:
pandas_gbq.to_gbq(dramitems_clean,table_id,project_id=gbq_proj_id,if_exists="replace")





100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 2949.58it/s]


# Querying our Data

In [41]:
# Your code here 

query = """
    SELECT COUNT(*)
    FROM `msba-project-2022.dram_shop.dram_items_2022`
"""

results = client.query(query)


for row in results :
    print(f'There were {row[0]} rows.')


There were 110836 rows.


# Querying to Dataframe

In [42]:
# Your code here 

query = """
    SELECT customer_name, customer_id, SUM(gross_sales) AS gross_sales
    FROM `msba-project-2022.dram_shop.dram_items_2022`
    WHERE customer_name != ""
    GROUP BY customer_name, customer_id
    ORDER BY gross_sales DESC
"""

customer_spend = pandas_gbq.read_gbq(query,project_id = gbq_proj_id)

#customer_spend.head()

Downloading: 100%|█████████████████████████████████████████████████████████████|


In [43]:
customer_spend.describe()

,gross_sales
count,14507.000000
mean,41.689854
std,94.357769
min,1.000000
25%,11.000000
50%,20.000000
75%,39.000000
max,2942.900000


Finally I'll create a query that finds sales by day by category for a certain location, and stores the results in a dataframe.

In [28]:
#write my query

query1 = """
        SELECT date, category, sum(gross_sales) as gross_sales
        FROM `msba-project-2022.dram_shop.dram_items_2022`
        WHERE location = "The Dram Shop Front St."
        GROUP BY date,category
    
        """

#name my dataframe
daily_category_sales = pandas_gbq.read_gbq(query1,project_id = gbq_proj_id)


Downloading: 100%|██████████| 5550/5550 [00:00<00:00, 10127.06rows/s]


In [29]:
category_summary = daily_category_sales.groupby('category').sum('gross_sales').nlargest(10,'gross_sales').reset_index()
category_summary

,category,gross_sales
0,F-IPA Draught,106691.75
1,F-Lagers/Pils/Wheat Draught,49730.00
2,F-Wine Draught,30619.75
3,F-Cider Draught,25524.25
4,F-Sour Draught,21662.00
5,F-Amber/Pale Draught,20298.50
6,F-Seasonal,16524.25
7,F-Porter/Stout Draught,13548.75
8,BTG Features!,10450.50
9,Kombucha,9671.50
